KNN CLASSIFICATION

In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

Initialise Elasticsearch

In [2]:
# Connect to Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Apply transport options to the Elasticsearch object
es.options(request_timeout=60, max_retries=5, retry_on_timeout=True)

# Define Elasticsearch index
index_name = "flow_data_enc"

Récupération des données depuis Elasticsearch

In [5]:
query = {
    "bool": {
        "should": [
            {"term": {"appName_HTTPWeb": True}},
            {"term": {"appName_SSH": True}},
        ],
        "minimum_should_match": 1,
    }
}

result = es.search(
    index=index_name,
    query=query,
    scroll="2m",
    size=10000,
)

# Initialize empty DataFrames for HTTPWeb, SSH, and Errors
df_httpweb = pd.DataFrame()
df_ssh = pd.DataFrame()
df_errors = pd.DataFrame()

# Continue scrolling until no more results
while len(result["hits"]["hits"]) > 0:
    # Process the current batch of results and create the DataFrame
    df = pd.DataFrame(hit["_source"] for hit in result["hits"]["hits"])

    # Convert as boolean (if NaN, then False)
    # df = df.fillna(False).astype(bool)

    # Separate the results based on appName_HTTPWeb and appName_SSH values
    httpweb_condition = df["appName_HTTPWeb"] == True
    ssh_condition = df["appName_SSH"] == True

    # Concatenate into respective DataFrames
    df_httpweb = pd.concat([df_httpweb, df[httpweb_condition]])
    df_ssh = pd.concat([df_ssh, df[ssh_condition]])
    df_errors = pd.concat([df_errors, df[~(httpweb_condition | ssh_condition)]])

    # Use the scroll ID to retrieve the next batch
    result = es.scroll(scroll_id=result["_scroll_id"], scroll="2m")

# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

# Print the shapes of the resulting DataFrames
print("HTTPWeb DataFrame shape:", df_httpweb.shape)
print("SSH DataFrame shape:", df_ssh.shape)
print("Errors DataFrame shape:", df_errors.shape)
print(df_httpweb.head())

KeyError: 'appName_SSH'